# Запросы к базе данных SQL

В данной работе предоставлена база данных, содержащая информацию о фондах и инвестициях. Задача состоит в выгрузке из БД информации, отвечающей требованиям запроса. ER-диаграмма БД выглядит следующим образом:

![1](Structure.png)

- `acquisition`: cодержит информацию о покупках одних компаний другими;
- `education`:содержит информацию об уровне образования сотрудников компаний;
- `people`: cодержит информацию о сотрудниках компаний-стартапов;
- `company`: содержит информацию о компаниях-стартапах;
- `investment`: содержит информацию об инвестициях венчурных фондов в компании-стартапы;
- `funding_round`: содержит информацию о раундах инвестиций;
- `fund`: хранит информацию о венчурных фондах.

Более подробно ознакомиться со структурой БД можно по следующей ссылке: https://code.s3.yandex.net/SQL%20for%20data%20and%20analytics/ER/basic_sql_project.pdf

### Задача №1: 

Необходимо посчитать, сколько компаний закрылось.

```sql
SELECT COUNT(*)
FROM company
WHERE status='closed';
```

### Задача №2:

Необходимо отобразить количество привлечённых средств для новостных компаний США. Полученный результат необходимо отсортировать по убыванию значений в поле `funding_total`.

```sql
SELECT funding_total
FROM company
WHERE category_code='news'
  AND country_code='USA'
ORDER BY funding_total DESC;
```

### Задача №3:

Необходимо найти общую сумму сделок по покупке одних компаний другими в долларах. Из полученного результата необходимо оставить только те сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

```sql
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code='cash'
  AND acquired_at BETWEEN '2011-01-01' AND '2013-12-31';
```

### Задача №4:

Необходимо отобразить имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.

```sql
SELECT first_name,
       last_name,
       twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%';
```

### Задача №5:

Необходимо отобразить всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку 'money', а фамилия начинается на 'K'.

```sql
SELECT *
FROM people
WHERE twitter_username LIKE '%money%'
 AND last_name LIKE 'K%';
```

### Задача №6:

Необходимо отобразить общую сумму привлечённых инвестиций для каждой страны, которые получили компании, зарегистрированные в этой стране. Полученные данные необходимо отсортировать по убыванию суммы.

```sql
SELECT country_code,
       SUM(funding_total) AS total_sum
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC;
```

### Задача №7:

Необходимо составить таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату. В итоговой таблице необходимо оставить только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

```sql
WITH tmp AS (
    SELECT
        funded_at,
        MIN(raised_amount) AS min_amount,
        MAX(raised_amount) AS max_amount
    FROM
        funding_round
    GROUP BY
        funded_at
)
SELECT *
FROM
    tmp
WHERE
    min_amount <> 0
    AND min_amount <> max_amount;
```

### Задача №8:

Необходимо отобразить все поля таблицы `fund`, а также поле с категориями:
- для фондов, которые инвестируют в 100 и более компаний - категория `high_activity`;
- для фондов, которые инвестируют в 20 и более компаний до 100 - категория `middle_activity`;
- если количество инвестируемых компаний фонда не достигает 20 - категория `low_activity`.


```sql
SELECT *,
       CASE
           WHEN invested_companies >= 100 THEN 'high_activity'
           WHEN invested_companies >= 20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity_level
FROM fund;
```

### Задача №9:

Для каждой из категорий `activity_level` из прошлого задания необходимо посчитать округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Необходимо вывести категории и среднее число инвестиционных раундов, а также отсортировать полученную таблицу по возрастанию среднего.

```sql
WITH tmp AS (
    SELECT *,
           CASE
               WHEN invested_companies >= 100 THEN 'high_activity'
               WHEN invested_companies >= 20 THEN 'middle_activity'
               ELSE 'low_activity'
           END AS activity_level
    FROM fund
)
SELECT
    activity_level,
    ROUND(AVG(investment_rounds)) AS avg_investment_rounds
FROM
    tmp
GROUP BY
    activity_level
ORDER BY
    avg_investment_rounds;
```

### Задача №10:

Необходимо проанализировать, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 

Для каждой страны необходимо посчитать минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Также необходимо исключить страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. 

В итоговую таблицу должны попасть десять самых активных стран-инвесторов. Таблицу необходимо отсортировать по среднему количеству компаний от большего к меньшему, а затем добавить сортировку по коду страны в лексикографическом порядке.

```sql
WITH tmp AS (
    SELECT
        country_code,
        MIN(invested_companies) AS min_invested_companies,
        MAX(invested_companies) AS max_invested_companies,
        AVG(invested_companies) AS avg_invested_companies
    FROM
        fund
    WHERE
        founded_at BETWEEN '2010-01-01' AND '2012-12-31'
    GROUP BY
        country_code
    HAVING
        MIN(invested_companies) <> 0
    ORDER BY
        avg_invested_companies DESC, country_code
    LIMIT 10
)
SELECT *
FROM
    tmp;
```

### Задача №11:

Необходимо отобразить имя и фамилию всех сотрудников стартапов. К полученной таблице необходимо добавить поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

```sql
SELECT p.first_name,
       p.last_name,
       e.instituition
FROM people AS p
LEFT JOIN education AS e ON p.id=e.person_id
```

### Задача №12:

Для каждой компании необходимо найти количество учебных заведений, которые окончили её сотрудники. Необходимо вывести название компании и число уникальных названий учебных заведений. Необходимо составить топ-5 компаний по количеству университетов.

```sql
SELECT c.name,
       COUNT(DISTINCT e.instituition) AS unique_inst_count
FROM company AS c
JOIN people AS p ON c.id=p.company_id
JOIN education AS e ON p.id=e.person_id
GROUP BY c.name
ORDER BY unique_inst_count DESC
LIMIT 5
```

### Задача №13:

Необходимо составить список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

```sql
WITH tmp AS (
    SELECT company_id
    FROM funding_round
    WHERE is_first_round = 1 AND is_last_round = 1
)
SELECT DISTINCT c.name
FROM company AS c
JOIN tmp ON c.id = tmp.company_id
WHERE c.status = 'closed';
```

### Задача №14:

Необходимо составить список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

```sql
WITH tmp AS (
    SELECT DISTINCT c.id
    FROM company AS c
    JOIN (
        SELECT company_id
        FROM funding_round
        WHERE is_first_round = 1 AND is_last_round = 1
    ) AS tmp_funding ON c.id = tmp_funding.company_id
    WHERE c.status = 'closed'
)
SELECT DISTINCT p.id
FROM people AS p
JOIN tmp ON p.company_id = tmp.id;
```

### Задача №15:

Необходимо составить таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

```sql
WITH tmp AS (
  SELECT DISTINCT c.name,
                  c.id
  FROM company AS c
  JOIN (
    SELECT company_id
    FROM funding_round
    WHERE is_first_round = 1 AND is_last_round = 1
  ) AS tmp_funding ON c.id = tmp_funding.company_id
  WHERE c.status = 'closed'
)
SELECT DISTINCT p.id,
       e.instituition
FROM people AS p
JOIN tmp ON tmp.id=p.company_id
JOIN education AS e ON p.id=e.person_id
GROUP BY p.id, e.instituition
```

### Задача №16:

Необходимо посчитать количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте необходимо учитывать, что некоторые сотрудники могли окончить одно и то же заведение дважды.

```sql
WITH tmp AS (
  SELECT DISTINCT c.name,
                  c.id
  FROM company AS c
  JOIN (
    SELECT company_id
    FROM funding_round
    WHERE is_first_round = 1 AND is_last_round = 1
  ) AS tmp_funding ON c.id = tmp_funding.company_id
  WHERE c.status = 'closed'
)
SELECT DISTINCT p.id,
       COUNT(e.instituition) AS inst_count
FROM people AS p
JOIN tmp ON tmp.id=p.company_id
JOIN education AS e ON p.id=e.person_id
GROUP BY p.id 
```

### Задача №17:

Необходимо вывести среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook.

```sql
WITH tmp AS (
    WITH fb AS (
        SELECT DISTINCT c.name,
                        c.id
        FROM company AS c
        WHERE c.name = 'Facebook'
    )
    SELECT DISTINCT p.id,
           COUNT(e.instituition)
    FROM people AS p
    JOIN fb ON fb.id = p.company_id
    JOIN education AS e ON p.id = e.person_id
    GROUP BY p.id
)
SELECT AVG(count)
FROM tmp;
```

### Задача №18:

Необходимо составить таблицу из полей:
- `name_of_fund` — название фонда;
- `name_of_company` — название компании;
- `amount` — сумма инвестиций, которую привлекла компания в раунде.

В таблицу должны войти данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

```sql
SELECT f.name AS name_of_fund,
       c.name AS name_of_company,
       fr.raised_amount
FROM investment AS i
JOIN fund AS f ON f.id=i.fund_id
JOIN funding_round AS fr ON fr.id=i.funding_round_id
JOIN company AS c ON c.id=i.company_id
WHERE c.milestones>6
  AND fr.funded_at BETWEEN '2012-01-01' AND '2013-12-31'
```

### Задача №19:

Необходимо отобрать данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Данные должны быть сгруппированы по номеру месяца; в финальную таблицу должны войти следующие поля:
- номер месяца, в котором проходили раунды;
- количество уникальных названий фондов из США, которые инвестировали в этом месяце;
- количество компаний, купленных за этот месяц;
- общая сумма сделок по покупкам в этом месяце.

```sql
WITH fd AS (
    SELECT EXTRACT(MONTH FROM fr.funded_at) AS month,
           COUNT(DISTINCT f.name) AS num_fund
    FROM fund AS f
    JOIN investment AS i ON f.id = i.fund_id
    JOIN funding_round AS fr ON fr.id = i.funding_round_id
    WHERE f.country_code = 'USA' AND EXTRACT(YEAR FROM fr.funded_at) BETWEEN 2010 AND 2013
    GROUP BY month
),

ad AS (
    SELECT EXTRACT(MONTH FROM acquired_at) AS month,
           COUNT(acquired_company_id) AS num_acq,
           SUM(price_amount) AS sum
    FROM acquisition
    WHERE EXTRACT(YEAR FROM acquired_at) BETWEEN 2010 AND 2013
    GROUP BY month
)

SELECT fd.month,
       fd.num_fund,
       ad.num_acq,
       ad.sum
FROM fd
JOIN ad ON fd.month = ad.month;
```

### Задание №20:

Необходимо составить сводную таблицу и вывести среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Необходимо отсортировать таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

```sql
WITH a_2011 AS (
    SELECT country_code,
           AVG(funding_total) AS avg
    FROM company
    WHERE EXTRACT(YEAR FROM founded_at) = 2011
    GROUP BY country_code
),

a_2012 AS (
    SELECT country_code,
           AVG(funding_total) AS avg
    FROM company
    WHERE EXTRACT(YEAR FROM founded_at) = 2012
    GROUP BY country_code
),

a_2013 AS (
    SELECT country_code,
           AVG(funding_total) AS avg
    FROM company
    WHERE EXTRACT(YEAR FROM founded_at) = 2013
    GROUP BY country_code
)

SELECT a_2011.country_code, a_2011.avg AS avg_2011, a_2012.avg AS avg_2012, a_2013.avg AS avg_2013
FROM a_2011
JOIN a_2012 ON a_2011.country_code = a_2012.country_code
JOIN a_2013 ON a_2011.country_code = a_2013.country_code
ORDER BY avg_2011 DESC;
```